In [1]:
import gurobipy as gp
import numpy as np
import ciropt as co
import sympy as sp

In [2]:
L_smooth = 1.
mu = 0.
Capacitance = 1.

time_limit = 300
ps = 10
psm = 1
heur = 0.001

solver = "gp"

In [3]:
def gp_print_solutions(model, gp_vars):
    for i in range(model.SolCount):
        model.Params.SolutionNumber = i
        # print(ns, gp_vars)
        print(f"{i+1}: obj = {model.PoolObjVal}")
        print(model.printQuality())
        # print(gp_vars["b"].Xn, gp_vars["h"].Xn,  gp_vars["alpha"].Xn,  gp_vars["beta"].Xn)
        print(co.dict_parameters_ciropt_gp(model, gp_vars, all=True, Xn=True))
    model.Params.SolutionNumber = 0
    model.update()

In [4]:
problem = co.gradient_flow_circuit(mu, L_smooth, Capacitance)
problem.obj = problem.b
res, co_model, sp_exp = problem.solve(solver=solver, verbose=False, debug=True, time_limit=time_limit, psm=psm, ps=ps)
gp_vars = problem.vars
print(res)

dim_G=5, dim_F=4
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-31
{'b': 1.0000004010112338, 'h': 1.000057693928346, 'd': 0.0, 'alpha': 7.794641180673609e-08, 'beta': 0.7468309943392725}


In [5]:
co_vars = co.dict_parameters_ciropt_gp(co_model, gp_vars, all=True)

In [6]:
# co_vars

In [7]:
np.linalg.norm((co.gurobi_to_numpy(problem.gp_expressions["sum_ij_La"])-co.gurobi_to_numpy(problem.gp_expressions["Fweights_d"])).flatten()), \
np.linalg.norm((co.gurobi_to_numpy(problem.gp_expressions["sum_ij_AC"]) - co.gurobi_to_numpy(problem.gp_expressions["Gweights_d"]) - co_vars["P"] @ co_vars["P"].T).flatten())

(5.858258176466979e-13, 4.988086185498642e-07)

In [8]:
size_I = 4          # I = { star, 1, 1.5, 2 }
dim_G = 5           # { x_star, x_1, y_1, y_1p5, y_2 }
dim_F = 4           # { f_star, f_1, f_1p5, f_2 }
model = gp.Model()

b = model.addVar(name='b', lb=0., ub=10)
h = model.addVar(name='h', lb=0., ub=10)
alpha = model.addVar(name='alpha', lb=-100, ub=100) #lb=-1.*gp.GRB.INFINITY, ub=gp.GRB.INFINITY)
beta = model.addVar(name='beta', lb=-100, ub=100) #lb=-1.*gp.GRB.INFINITY, ub=gp.GRB.INFINITY)
alpha_h = model.addVar(name='alpha_h', lb=-1000, ub=1000)
beta_h = model.addVar(name='beta_h', lb=-1000, ub=1000)
P = model.addMVar((dim_G, dim_G), name='P', lb=-1.*gp.GRB.INFINITY, ub=gp.GRB.INFINITY)
lamb0 = model.addMVar((size_I, size_I), name='lamb0', lb=0, ub=gp.GRB.INFINITY)
model.update()

y_star = np.zeros(dim_G).reshape(-1, 1)
y_1 = co.one_hot(dim_G, 2)
y_1p5 = co.one_hot(dim_G, 3) 
y_2 = co.one_hot(dim_G, 4) 
Gs = [y_star, y_1, y_1p5, y_2]

f_star = co.one_hot(dim_F, 0) # np.zeros((dim_F, 1))
f_1   = co.one_hot(dim_F, 1) 
f_1p5 = co.one_hot(dim_F, 2)
f_2   = co.one_hot(dim_F, 3)
F = [f_star, f_1, f_1p5, f_2]

x_star = co.one_hot(dim_G, 0)
x_1    = co.one_hot(dim_G, 1)
x_1p5 = x_1 - (alpha_h / Capacitance) * y_1
x_2   = x_1 - (beta_h / Capacitance) * y_1 - ((h - beta_h) / Capacitance) * y_1p5 
Xs = [x_star, x_1, x_1p5, x_2]

sp_vars = {name : sp.symbols(name) for name in gp_vars.keys()}
sp_x_1p5 = x_1 - (sp_vars["alpha_h"] / Capacitance) * y_1
sp_x_2   = x_1 - (sp_vars["beta_h"] / Capacitance) * y_1 - ((sp_vars["h"] - sp_vars["beta_h"]) / Capacitance) * y_1p5 
sp_Xs = [x_star, x_1, sp_x_1p5, sp_x_2]

star_idx = 4
A = lambda i,j: co.symm_prod(Gs[j], Xs[i] - Xs[j])
B = lambda i,j: co.symm_prod(Xs[i] - Xs[j], Xs[i] - Xs[j])
C = lambda i,j: co.symm_prod(Gs[i] - Gs[j], Gs[i] - Gs[j])
a = lambda i,j: F[j] - F[i]

sp_A = lambda i,j: co.symm_prod(Gs[j], sp_Xs[i] - sp_Xs[j])
sp_B = lambda i,j: co.symm_prod(sp_Xs[i] - sp_Xs[j], sp_Xs[i] - sp_Xs[j])
sp_C = lambda i,j: co.symm_prod(Gs[i] - Gs[j], Gs[i] - Gs[j])

assert (dim_G, dim_G) == A(1, 2).shape == B(1, 2).shape == C(1, 2).shape
assert a(1, 2).shape == (dim_F, 1)

model.addConstr( alpha_h == alpha * h )
model.addConstr( beta_h == beta * h )
model.addConstr( lamb0.diagonal() == np.zeros(size_I) )
model.addConstr( P.diagonal() >= np.zeros(dim_G) )
for i in range(dim_G):
            # model.addConstr( P[i, i] >= 0 )
            for j in range(i+1, dim_G):
                model.addConstr( P[i, j] == 0 )

model.setObjective( -b )
# star:0, 1:1, 1p5:2, 2:3
sum_ij_La = np.zeros((dim_F, 1), dtype=object)
sum_ij_AC  = np.zeros((dim_G, dim_G), dtype=object)
for i in range(size_I):
    for j in range(size_I):
        if i == j: continue
        sum_ij_La += lamb0[i, j].item() * a(i, j)
        sum_ij_AC += lamb0[i, j].item() * (A(i, j) + (1./(2 * L_smooth)) * C(i, j))

Fweights_d = - b * a(1, 0)
Gweights_d = (Capacitance / 2) * (B(3, 0) - B(1, 0))
# model.addConstr( sum_ij_La == Fweights_d ) 
# print(sum_ij_La, "\n", Fweights_d)
# model.addConstr( sum_ij_AC  - P @ P.T - Gweights_d == np.zeros((dim_G, dim_G)))
z1 = sum_ij_La - Fweights_d
z2 = sum_ij_AC - Gweights_d
PPt = P @ P.T
model.addConstrs(sum_ij_La[i, 0] - Fweights_d[i, 0] == 0 for i in range(dim_F))
model.addConstrs(sum_ij_AC[i, j] - Gweights_d[i, j] - PPt[i, j].item() == 0 for i in range(dim_G) for j in range(dim_G))

model.params.NonConvex = 2
model.params.TimeLimit = time_limit
model.params.PoolSolutions = ps
model.Params.PoolSearchMode = 1
# model.Params.FeasibilityTol = 1e-9
model.Params.Heuristics = heur
model.Params.LogToConsole = 0
model.update()
model.optimize()
print(model.printQuality())

Set parameter NonConvex to value 2
Set parameter TimeLimit to value 300
Set parameter PoolSearchMode to value 1
Set parameter Heuristics to value 0.001
None


In [9]:
np.linalg.norm((co.gurobi_to_numpy(sum_ij_AC) - co.gurobi_to_numpy(Gweights_d) - P.X @ P.X.T).flatten())

2.777164555957505e-07

In [10]:
gram_gp_vars = {'b': b,
                'h': h,
                'alpha': alpha,
                'beta': beta, 
                'P': P, 
                'lamb0': lamb0}

In [11]:
# for name, val in co_vars.items():
#     if type(val) == np.ndarray:
#         print(name, val.min(), val.max())
#     else:
#         print(name, val)

In [12]:
sol = co.dict_parameters_ciropt_gp(model, gram_gp_vars, all=True)
sol

{'b': 1.000000092397201,
 'h': 1.0005480447881754,
 'alpha': -100.0,
 'beta': 1.0,
 'P': array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        , -0.00044314,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]]),
 'lamb0': array([[-0.00000000e+00,  1.00054804e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 5.47952391e-04, -0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         -0.00000000e+00]])}

In [13]:
# gp_print_solutions(model, gram_gp_vars)

In [14]:
# compare matrices
size_I_function = size_I
for i in range(size_I_function):
    for j in range(size_I_function):
        if i == j: continue
        F1, G1 = sp_exp[(0,i,j)]["F"], sp_exp[(0,i,j)]["G"]
        F2 = a(i, j)
        G2 = sp_A(i, j) + (1./(2*L_smooth)) * sp_C(i, j)
        assert co.equal_sp_arrays(G1, G2), print(f"{i=}, {j=} \n{G1=} \n{G2=}")
        assert co.equal_sp_arrays(F1, F2[:, 0]), print(f"{i=}, {j=} \n{F1=} \n{F2=}\n")

F1, G1 = sp_exp["FG_d"]["F"], sp_exp["FG_d"]["G"]
F2 = -sp_vars["b"] * a(1, 0)
G2 = (Capacitance / 2) * (sp_B(3, 0) - sp_B(1, 0))
assert co.equal_sp_arrays(G1, G2), print(f"{G1=} \n{G2=}")
assert co.equal_sp_arrays(F1, F2[:,0]), print(f"{F1=} \n{F2=}\n")

In [15]:
problem.model.printQuality(), model.printQuality()

(None, None)

In [16]:
# def evaluate_model_vals(model, names, vars, vals):
#     for name in names:
#         v = vars[name]
#         v_val = vals[name].X
#         v.setAttr(gp.GRB.Attr.LB, v_val) 
#         v.setAttr(gp.GRB.Attr.UB, v_val)
#     model.update()
#     model.optimize()

# main_names = ["h", "b", "alpha", "beta", "P", "lamb0"]
# evaluate_model_vals(problem.model, main_names, \
#                     vars=gp_vars, vals=gram_gp_vars)


# for name in main_names:
#     v_eval = gp_vars[name].X
#     assert np.allclose(v_eval, gram_gp_vars[name].X), print(name, v_eval, gram_gp_vars[name].X)

# for name in ['sum_ij_AC', 'sum_ij_La', 'Fweights_d', 'Gweights_d']:
#     v_eval = co.gurobi_to_numpy(problem.gp_expressions[name]) 
#     v_eval_gram = co.gurobi_to_numpy(globals()[name])
#     assert np.allclose(0, np.linalg.norm(v_eval.flatten()-v_eval_gram.flatten())), print(name)

# sum1 = co.gurobi_to_numpy(problem.gp_expressions['sum_ij_La'] - \
#                           problem.gp_expressions['Fweights_d'])
# sum2 = co.gurobi_to_numpy(problem.gp_expressions['sum_ij_AC'] - \
#                           problem.gp_expressions['Gweights_d']) - \
#                           gp_vars['P'].X @ gp_vars['P'].X.T

# assert np.allclose(0, np.linalg.norm(sum1.flatten())) and np.allclose(0, np.linalg.norm(sum2.flatten()))

# print("PASSED")

In [17]:
h_init = co_vars["h"]
b_init = co_vars["b"]
alpha_init = co_vars["alpha"]
beta_init = co_vars["beta"]
lamb_init = co_vars["lamb0"]
P_init = co_vars["P"]
alpha_h_init = alpha_init * h_init
beta_h_init = beta_init * h_init

In [18]:
model.NumVars, model.objVal

(47, -1.000000092397201)

In [19]:
gp_vars.keys()

dict_keys(['b', 'd', 'h', 'alpha', 'beta', 'P', 'lamb0', 'beta_h', 'alpha_h', 'beta_beta', 'beta_beta_h', 'beta_beta_h_h', 'beta_h_h', 'h_h'])

In [20]:
vars = [b, h, alpha, beta, alpha_h, beta_h, P, lamb0]
eval_vars = [b_init, h_init, alpha_init, beta_init, alpha_h_init, beta_h_init, P_init, lamb_init]

for v, v_val in zip(vars, eval_vars):
    v.setAttr(gp.GRB.Attr.LB, v_val) 
    v.setAttr(gp.GRB.Attr.UB, v_val)
model.update()
model.optimize()


b.X, h.X, alpha.X, beta.X

(1.0000004010112338,
 1.000057693928346,
 7.794641180673609e-08,
 0.7468309943392725)

In [22]:
for name in ['h', 'b', 'alpha', 'beta', 'P', 'lamb0']:
    v_eval = globals()[name].X
    assert np.allclose(v_eval, co_vars[name]), print(name, v_eval, co_vars[name])

for name in ['sum_ij_AC', 'sum_ij_La', 'Fweights_d', 'Gweights_d']:
    v_eval = co.gurobi_to_numpy(globals()[name])
    assert np.allclose(0, np.linalg.norm(v_eval.flatten()-\
            co.gurobi_to_numpy(problem.gp_expressions[name]).flatten())), print(name)

sum1 = co.gurobi_to_numpy(sum_ij_La - Fweights_d).flatten()
sum2 = (co.gurobi_to_numpy(sum_ij_AC - Gweights_d) - co.gurobi_to_numpy(PPt)).flatten()

co_sum1 = (co.gurobi_to_numpy(problem.gp_expressions["sum_ij_La"])-co.gurobi_to_numpy(problem.gp_expressions["Fweights_d"])).flatten()
co_sum2 = (co.gurobi_to_numpy(problem.gp_expressions["sum_ij_AC"]) - co.gurobi_to_numpy(problem.gp_expressions["Gweights_d"]) - co_vars["P"] @ co_vars["P"].T).flatten()

# assert np.allclose(0, np.linalg.norm(sum1.flatten())) and np.allclose(0, np.linalg.norm(sum2.flatten()))
assert np.allclose(0, np.linalg.norm(sum1-co_sum1)) and np.allclose(0, np.linalg.norm(sum2-co_sum2))

print("PASSED")

PASSED
